# Chapter 11-04: 커널 프로파일링과 Kernel Fusion 전략

## 학습 목표
- **nvprof / Nsight Systems**의 측정 지표(SM 점유율, 메모리 대역폭, Warp Efficiency)를 해석하는 방법을 이해한다
- 연산 그래프에서 **Fusion 가능 쌍**을 식별하고, Elementwise Fusion / Reduction Fusion의 원칙을 적용한다
- **LayerNorm Fusion** 사례를 통해 표준 구현 대비 HBM 접근 횟수 절약을 수식으로 정량화한다
- Python 수준에서 임의 연산 시퀀스의 Fused 구현과 Unfused 구현 성능을 측정하고 비교한다

## 목차
1. [수학적 기초: Fusion 유형과 메모리 절약 공식](#1.-수학적-기초)
2. [프로파일링 지표 해석 가이드](#2.-프로파일링-지표)
3. [Elementwise Fusion 구현](#3.-Elementwise-Fusion)
4. [LayerNorm Fusion: 수식 도출 + 구현](#4.-LayerNorm-Fusion)
5. [Fusion 결정 트리 및 전략](#5.-Fusion-전략)
6. [Chapter 11 종합 정리](#6.-Chapter-11-종합정리)

## 1. 수학적 기초

### Kernel Fusion 유형별 메모리 절약

**Elementwise Fusion** (Add → Dropout → GeLU → Scale):

$$M_{unfused} = (k_{ops} + 1) \cdot 2S \text{ bytes} \quad \text{(k개 연산 각각 읽기+쓰기)}$$
$$M_{fused} = 2S \text{ bytes} \quad \text{(1회 읽기 + 1회 쓰기)}$$
$$\text{절약} = k_{ops} \text{배}$$

**Layernorm Fusion** (입력 $x \in \mathbb{R}^H$ 기준):

표준 (3-pass):
1. $\mu = \frac{1}{H} \sum x_i$ (Read 1)
2. $\sigma^2 = \frac{1}{H} \sum (x_i - \mu)^2$ (Read 2)
3. $y_i = \gamma \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$ (Read 3, Write 1)

Fused (2-pass 또는 Welford 알고리즘 1-pass):
$$\mu, \sigma^2 \text{를 Welford 방식으로 1-pass에서 동시 계산} \Rightarrow \text{총 2-pass}$$

### Welford Online 평균/분산 알고리즘

$$n_j = j, \quad \delta_j = x_j - \mu_{j-1}$$
$$\mu_j = \mu_{j-1} + \frac{\delta_j}{n_j}, \quad M_{2j} = M_{2, j-1} + \delta_j (x_j - \mu_j)$$
$$\sigma^2 = \frac{M_{2N}}{N}$$

---

### 🐣 초등학생을 위한 친절 설명!

#### 🚂 Kernel Fusion이 기차역 비유

> 서울 → 대전 → 대구 → 부산을 간다면:
> - **Unfused**: 서울에서 집 가방을 꺼내고 → 대전에서 집→대전 가방 교체 → ...
> - **Fused (직행)**: 서울에서 가방 한 번 들고 → 부산까지 직행!
>
> 메모리 접근 = 역마다 가방을 풀고 싸는 시간  
> Fusion = 출발지에서 한 번만 꺼내고 목적지에서 한 번만 넣기

### 📝 연습 문제

#### 문제 1: Elementwise Fusion 절약 계산

Attention 출력 후 Add(Residual) + Dropout + LayerNorm 의 3개 연산을 Fuse할 때,  
$S = 2048 \times 4096$ (모든 토큰의 Hidden) FP16 기준 HBM 절약량은?

<details>
<summary>💡 풀이 확인</summary>

$S_{total} = 2048 \times 4096 = 8.39M$ 원소, FP16 = 2 bytes

- **Unfused**: 3 연산 각각 읽기+쓰기 = $6 \times 8.39M \times 2 = 100.7$ MB
- **Fused**: 읽기 1 + 쓰기 1 = $2 \times 8.39M \times 2 = 33.6$ MB  
- 절약: **3배**, 67.1 MB 절감
</details>

#### 문제 2: Fusion 가능/불가 판별

다음 연산 쌍에서 Fusion 가능한 것은? 이유를 설명하라.
- (A) GeLU → LayerNorm
- (B) GEMM → Add + ReLU
- (C) Softmax → Dropout
- (D) Cross-Entropy Loss → Backward Pass

<details>
<summary>💡 풀이 확인</summary>

- **(A) O**: GeLU는 Elementwise → LN은 같은 텐서에 대해 Reduction. 쌍은 가능 (예: Apex FusedGeLU + LN)
- **(B) ✅ 가장 일반적**: GEMM 후 Activation은 Epilogue Fusion (cuBLAS 지원), Add+ReLU는 Elementwise라 즉시 Fuse
- **(C) ✅**: 둘 다 Elementwise 수준 (Softmax 출력에 Dropout 적용), FlashAttention이 이를 포함
- **(D) ❌ 어려움**: Backward는 Forward의 Activation 저장이 필요 → 별도 커널로 분리하는 것이 일반적
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

np.random.seed(42)

## 2. 프로파일링 지표

In [ ]:
# ---------------------------------------------------
# GPU 프로파일링 지표 해석 가이드 (시뮬레이션 데이터)
# ---------------------------------------------------

# 실제 Nsight Systems 출력에서 볼 수 있는 지표들
profile_guide = [
    ('SM Util (%)',         'SM 활성 비율 (이상: ≥80%)',          '높을수록 좋음',     'block size↑ or grid 최적화'),
    ('Memory Util (%)',     'HBM 대역폭 활용률 (이상: ≥80%)',    '높을수록 좋음',     'Coalesced Access 확인'),
    ('Warp Efficiency (%)', '비분기 Warp 비율',                   '높을수록 좋음',     'if 문 제거, branch divergence↓'),
    ('L1 Hit Rate (%)',     '1차 캐시 적중률',                     '높을수록 좋음',     'Shared Memory 활용 증가'),
    ('Shared Mem Usage',    '커널당 Shared Memory 사용량',         '최대한 활용',       'tiling block 크기 증가'),
    ('Register/Thread',     '스레드당 레지스터 사용량',            '256K/SM 내에서 최대', 'occupancy와 trade-off'),
    ('Kernel Duration (ms)', '커널 실행 시간',                    '최소화',            'FLOPs/ms = TFLOPS 계산'),
]

print("=" * 90)
print(f"  GPU 커널 프로파일링 지표 해석 가이드")
print("=" * 90)
print(f"{'지표':<25} {'설명':<35} {'목표':<15} {'최적화 힌트'}")
print("-" * 90)
for metric, desc, target, hint in profile_guide:
    print(f"{metric:<25} {desc:<35} {target:<15} {hint}")

# 시뮬레이션된 프로파일링 데이터로 시각화
kernels = {
    'Unfused GEMM':        {'SM': 82, 'MEM': 45, 'WARP_EFF': 97, 'L1': 35, 'ANALYSIS': 'Compute-bound ⚡'},
    'Unfused Softmax':     {'SM': 20, 'MEM': 88, 'WARP_EFF': 95, 'L1': 60, 'ANALYSIS': 'Memory-bound 🐌'},
    'Unfused LayerNorm':   {'SM': 18, 'MEM': 85, 'WARP_EFF': 92, 'L1': 55, 'ANALYSIS': 'Memory-bound 🐌'},
    'Fused Softmax+Drop':  {'SM': 22, 'MEM': 75, 'WARP_EFF': 94, 'L1': 72, 'ANALYSIS': 'Fused: BW↑'},
    'Fused LayerNorm':     {'SM': 25, 'MEM': 78, 'WARP_EFF': 93, 'L1': 80, 'ANALYSIS': 'Fused: BW↑'},
    'FlashAttention':      {'SM': 75, 'MEM': 65, 'WARP_EFF': 96, 'L1': 85, 'ANALYSIS': 'Optimal!'},
}

fig, ax = plt.subplots(figsize=(13, 5))
x = np.arange(len(kernels))
w = 0.22
metrics = ['SM', 'MEM', 'WARP_EFF', 'L1']
metric_labels = ['SM Util', 'Memory Util', 'Warp Eff', 'L1 Hit Rate']
colors_p = ['#1E88E5', '#E53935', '#43A047', '#FB8C00']

for i, (m, label, c) in enumerate(zip(metrics, metric_labels, colors_p)):
    vals = [kernels[k][m] for k in kernels]
    ax.bar(x + i*w, vals, w, label=label, color=c, alpha=0.80, edgecolor='white', lw=1)

ax.axhline(y=80, color='k', ls='--', lw=1.5, alpha=0.6, label='이상적 80% 기준선')
ax.set_xticks(x + w*1.5)
ax.set_xticklabels(list(kernels.keys()), rotation=20, ha='right', fontsize=9)
ax.set_ylabel('활용률 (%)', fontsize=11)
ax.set_title('GPU 커널 프로파일링 시뮬레이션 결과\n(Fused 버전이 전반적으로 더 균형 잡힌 성능)', fontweight='bold')
ax.legend(fontsize=9, loc='upper right'); ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3. Elementwise Fusion

In [ ]:
# ---------------------------------------------------
# Elementwise Kernel Fusion 구현 및 성능 측정
# ---------------------------------------------------

def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2/np.pi) * (x + 0.044715 * x**3)))

def unfused_ops(x, bias, residual, dropout_prob=0.1):
    """Unfused: 각 연산이 별도 메모리 읽기/쓰기 수행"""
    # Op 1: Bias Add
    y1 = x + bias                    # HBM 읽기 2 + 쓰기 1
    # Op 2: GeLU
    y2 = gelu(y1)                    # HBM 읽기 1 + 쓰기 1
    # Op 3: Residual Add
    y3 = y2 + residual               # HBM 읽기 2 + 쓰기 1
    # Op 4: Dropout (간단 버전)
    mask = np.random.rand(*y3.shape) > dropout_prob
    y4 = y3 * mask / (1 - dropout_prob)  # HBM 읽기 1 + 쓰기 1
    return y4


def fused_ops(x, bias, residual, dropout_prob=0.1):
    """Fused: 단일 패스에서 모든 연산 처리 (HBM 1회 읽기/쓰기)"""
    mask = np.random.rand(*x.shape) > dropout_prob
    # 모든 연산을 하나의 표현식으로 (단일 메모리 패스)
    y_fused = gelu(x + bias) + residual
    y_fused = y_fused * mask / (1 - dropout_prob)
    return y_fused


# 설정
S, H = 4096, 4096  # 시퀀스×히든
X  = np.random.randn(S, H).astype(np.float32)
b  = np.random.randn(H).astype(np.float32)
r  = np.random.randn(S, H).astype(np.float32)

n_runs = 50

# Unfused 측정
np.random.seed(0)
t0 = time.perf_counter()
for _ in range(n_runs):
    Y_u = unfused_ops(X, b, r)
t_unfused = (time.perf_counter() - t0) / n_runs * 1000

# Fused 측정
np.random.seed(0)
t0 = time.perf_counter()
for _ in range(n_runs):
    Y_f = fused_ops(X, b, r)
t_fused = (time.perf_counter() - t0) / n_runs * 1000

# HBM 접근량 이론값
dtype_bytes = 4  # FP32
n_elements  = S * H
hbm_unfused = (2+1 + 1+1 + 2+1 + 1+1) * n_elements * dtype_bytes  # 각 Op R+W
hbm_fused   = (1+1) * n_elements * dtype_bytes

print(f"=== Elementwise Fusion 성능 비교 (S={S}, H={H}) ===")
print(f"  Unfused 실행: {t_unfused:.2f} ms")
print(f"  Fused 실행:   {t_fused:.2f} ms")
print(f"  속도 향상:    {t_unfused/t_fused:.2f}x (CPU 기준)")
print()
print(f"  Unfused 이론 HBM: {hbm_unfused/1e6:.0f} MB")
print(f"  Fused 이론 HBM:   {hbm_fused/1e6:.0f} MB")
print(f"  이론 HBM 절약:    {hbm_unfused//hbm_fused}배 ({(1-hbm_fused/hbm_unfused)*100:.0f}%)")
print("  (실제 GPU에서는 이론값과 거의 일치)")

## 4. LayerNorm Fusion

In [ ]:
# ---------------------------------------------------
# Fused LayerNorm: Welford 알고리즘으로 1-pass
# ---------------------------------------------------

def layernorm_standard(x, gamma, beta, eps=1e-5):
    """표준 LayerNorm (3-pass): mean → variance → normalize"""
    # Pass 1: 평균
    mu = np.mean(x, axis=-1, keepdims=True)
    # Pass 2: 분산
    var = np.var(x, axis=-1, keepdims=True)
    # Pass 3: 정규화 + 스케일/이동
    return gamma * (x - mu) / np.sqrt(var + eps) + beta


def welford_online_stats(x):
    """
    Welford 알고리즘: 단일 패스로 평균과 분산을 동시 계산.
    수치적으로 안정적.
    
    재귀:
      n_j = j
      delta_j = x_j - mean_{j-1}
      mean_j = mean_{j-1} + delta_j / n_j
      M2_j = M2_{j-1} + delta_j * (x_j - mean_j)
      variance = M2_N / N
    """
    n, mean, M2 = 0, 0.0, 0.0
    for xi in x:
        n += 1
        delta = xi - mean
        mean += delta / n
        M2 += delta * (xi - mean)
    return mean, M2 / n  # (평균, 분산)


def layernorm_fused(x, gamma, beta, eps=1e-5):
    """Fused LayerNorm: Welford로 2-pass (stats 1-pass + normalize 1-pass)"""
    # Pass 1: Welford로 mean, var 동시 계산 (HBM 1회 읽기)
    mu  = np.mean(x, axis=-1, keepdims=True)   # NumPy는 내부적으로 2-pass이지만
    var = np.var(x, axis=-1, keepdims=True)    # CUDA Triton은 1-pass Welford 사용

    # Pass 2: 정규화 (HBM 1회 읽기 + 1회 쓰기)
    return gamma * (x - mu) / np.sqrt(var + eps) + beta


# Welford 알고리즘 검증
x_test = np.array([2.0, 4.0, 4.0, 4.0, 5.0, 5.0, 7.0, 9.0])
wf_mean, wf_var = welford_online_stats(x_test)
np_mean = np.mean(x_test)
np_var  = np.var(x_test)

print("[Welford 알고리즘 검증]")
print(f"  입력: {x_test}")
print(f"  Welford Mean: {wf_mean:.6f}  |  NumPy: {np_mean:.6f}  → 오차: {abs(wf_mean-np_mean):.2e}")
print(f"  Welford Var:  {wf_var:.6f}  |  NumPy: {np_var:.6f}  → 오차: {abs(wf_var-np_var):.2e}")
print(f"  {'✅ 수치적으로 동일' if abs(wf_var-np_var) < 1e-10 else '❌ 불일치'}")

# LayerNorm Fused vs Standard 검증
B, S, H = 4, 128, 512
X_ln = np.random.randn(B, S, H).astype(np.float64)
gamma = np.ones(H, dtype=np.float64)
beta  = np.zeros(H, dtype=np.float64)

Y_std   = layernorm_standard(X_ln, gamma, beta)
Y_fused = layernorm_fused(X_ln, gamma, beta)
err_ln  = np.abs(Y_std - Y_fused).max()

print(f"\n[LayerNorm Fused 검증] B={B}, S={S}, H={H}")
print(f"  최대 오차: {err_ln:.2e}  {'✅' if err_ln < 1e-10 else '❌'}")

# HBM 접근 비교
n_elem = B * S * H
hbm_std   = 3 * n_elem * 8  # 3읽기 + 중간 쓰기 포함 (대략)
hbm_fused = 2 * n_elem * 8  # 1읽기(stats) + 1읽기+쓰기(norm)
print(f"\nHBM 접근 비교 (FP64 기준):")
print(f"  표준 LN:  ~{hbm_std/1e6:.0f} MB")
print(f"  Fused LN: ~{hbm_fused/1e6:.0f} MB")
print(f"  절약:     ~{(1-hbm_fused/hbm_std)*100:.0f}%")

## 5. Fusion 전략

In [ ]:
# ---------------------------------------------------
# Fusion 종합 효과 시각화 및 결정 트리
# ---------------------------------------------------

print("=" * 65)
print("  Kernel Fusion 결정 트리")
print("=" * 65)
decision_tree = '''
연산 쌍 분석
    │
    ├─ Elementwise only? (ReLU, Add, Scale, Dropout...) 
    │       ──→ ✅ 항상 Fusion (HBM k배 절약)
    │
    ├─ Elementwise → Reduction? (→ Softmax, LayerNorm, Sum)
    │       ──→ ✅ Fusion 가능 (Triton, Flash Attention 사용)
    │
    ├─ GEMM → Elementwise? (GEMM + Bias + Activation)
    │       ──→ ✅ cuBLAS Epilogue Fusion 사용
    │
    ├─ 두 개의 독립된 Reduction?
    │       ──→ ⚠️  경우에 따라 (입력 공유 시 Fusion 가능)
    │
    └─ Backward Pass 포함?
            ──→ ❌ 일반적으로 어려움 (Activation 저장 필요)
                   → Gradient Checkpointing 대신 고려
'''
print(decision_tree)

# Transformer 레이어의 Fusion 적용 예
print("=== Transformer 레이어 Fusion 기회 분석 ===")
ops_in_transformer = [
    # (이름, Fusion 그룹, HBM 절약배수)
    ('QKV Projection (GEMM)',          'A', 1),
    ('Attention Score (GEMM)',          'A', 1),
    ('Softmax',                         'B', None),
    ('Dropout (Attn)',                  'B', None),
    ('→ Flash: Softmax+Dropout Fused', 'B', 3),
    ('Attention @ V (GEMM)',            'C', 1),
    ('Output Proj (GEMM)',              'D', 1),
    ('Add Residual 1',                  'E', None),
    ('LayerNorm 1',                     'E', None),
    ('→ Fused: Add+LN 1',             'E', 3),
    ('FFN GEMM 1',                      'F', 1),
    ('GeLU + Dropout (FFN)',            'G', None),
    ('→ Fused: GeLU+Drop',            'G', 2),
    ('FFN GEMM 2',                      'H', 1),
    ('Add Residual 2 + LN 2',          'I', None),
    ('→ Fused: Add+LN 2',             'I', 3),
]

for name, group, savings in ops_in_transformer:
    if savings is None:
        print(f"  {group}. {name}  ← 개별 커널")
    elif name.startswith('→'):
        print(f"      {name}  (HBM {savings}× 절약!) ✅")
    else:
        print(f"  {group}. {name}")

In [ ]:
# ---------------------------------------------------
# Fusion 전후 HBM 총 접근량 비교 시각화
# ---------------------------------------------------

# Transformer 1 레이어의 HBM 접근량 추정 (S=2048, H=4096, FP16)
S, H, d_head = 2048, 4096, 64
n_heads = H // d_head
n_elem_SH = S * H
n_elem_SS = S * S

# Unfused HBM (MB, FP16=2bytes)
ops_unfused = {
    'QKV Proj': 3 * 2 * n_elem_SH * 2 / 1e6,    # 3 GEMM
    'Attn Score': 2 * n_elem_SS * n_heads * 2 / 1e6,
    'Softmax': 5 * n_elem_SS * n_heads * 2 / 1e6,   # 5-pass
    'Dropout1': 3 * n_elem_SS * n_heads * 2 / 1e6,
    'Attn@V': 2 * n_elem_SH * 2 / 1e6,
    'Out Proj': 2 * n_elem_SH * 2 / 1e6,
    'LN1': 6 * n_elem_SH * 2 / 1e6,
    'FFN1': 2 * n_elem_SH * 4 * 2 / 1e6,        # 4H intermediate
    'GeLU': 3 * n_elem_SH * 4 * 2 / 1e6,
    'Drop2': 3 * n_elem_SH * 4 * 2 / 1e6,
    'FFN2': 2 * n_elem_SH * 4 * 2 / 1e6,
    'LN2': 6 * n_elem_SH * 2 / 1e6,
}

# Fused HBM
ops_fused = {
    'QKV Proj': ops_unfused['QKV Proj'],   # GEMM은 Fusion 효과 작음
    'FlashAttn': 2 * n_elem_SH * n_heads * 2 / 1e6 * 3,  # 크게 줄임
    'Out Proj': ops_unfused['Out Proj'],
    'Fused LN1': ops_unfused['LN1'] * 0.5,
    'FFN1': ops_unfused['FFN1'],
    'Fused GeLU+Drop': ops_unfused['GeLU'] * 0.5,
    'FFN2': ops_unfused['FFN2'],
    'Fused LN2': ops_unfused['LN2'] * 0.5,
}

total_unfused = sum(ops_unfused.values())
total_fused   = sum(ops_fused.values())

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for ax, data, title, c in [
    (axes[0], ops_unfused, f'Unfused HBM 총계: {total_unfused:.0f} MB', '#E53935'),
    (axes[1], ops_fused,   f'Fused HBM 총계: {total_fused:.0f} MB (↓{(1-total_fused/total_unfused)*100:.0f}%)', '#1E88E5'),
]:
    bars = ax.barh(list(data.keys()), list(data.values()), color=c, alpha=0.8, edgecolor='white')
    for bar, val in zip(bars, data.values()):
        ax.text(val + 5, bar.get_y() + bar.get_height()/2,
                f'{val:.0f} MB', va='center', fontsize=8)
    ax.set_xlabel('HBM 접근량 (MB)', fontsize=10)
    ax.set_title(title, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print(f"\nFusion 효과 요약:")
print(f"  Unfused HBM: {total_unfused:.0f} MB")
print(f"  Fused HBM:   {total_fused:.0f} MB")
print(f"  절약:        {total_unfused - total_fused:.0f} MB ({(1-total_fused/total_unfused)*100:.0f}%)")

## 6. Chapter 11 종합정리

### Chapter 11 전체 요약

| Chapter | 핵심 내용 | 핵심 지표 |
|---------|----------|----------|
| **11-01** | GPU 아키텍처, Roofline | AI = FLOPs/Bytes, Ridge ≈ 295 FLOPs/B (H100) |
| **11-02** | CUDA Tiled GEMM | Shared Memory로 T배 HBM 절약 |
| **11-03** | Triton, Online Softmax | Online Softmax: 1-pass, FlashAttn: $O(S^2) \to O(S)$ |
| **11-04** | Kernel Fusion 전략 | Elementwise k배, LN 2배, FlashAttn 크게 절약 |

### 커스텀 커널 최적화 원칙

```
1. 먼저 Roofline 분석 → Compute/Memory bound 판별
2. Memory-bound → Fusion 우선 (HBM 접근 최소화)
3. Compute-bound → Tiling (Shared Memory 활용)
4. 항상 프로파일링으로 검증 (추측하지 말고 측정!)
5. 라이브러리 우선 (cuBLAS, cuDNN) → 필요 시 Triton → 최후 CUDA C++
```

### 다음 챕터 예고
**Chapter 12: 최신 대형 언어 모델 아키텍처** — RoPE, SwiGLU, GQA, MoE 등 Llama-3와 GPT-4 계열 모델에서 사용하는 아키텍처 혁신들을 수학적으로 완전히 이해하고 직접 구현한다.